In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#!pip install pyprind

In [3]:
os.chdir('C:/Users/unlea/Desktop/Python 3/ML_Master/python-machine-learning-book-3rd-edition/ch08')
df = pd.read_csv('movie_data.csv',encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
df.shape

(50000, 2)

In [5]:
#文章の配列を、BoW表現にする
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()

#カウントベースのvectorizerに渡すとき、空白などで区切られた単語の配列になっている必要がある
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and The weather is sweet'
    
])
#fit_transformメソッドで、3つの文章を特徴量ベクトルに変換している
bag =CV.fit_transform(docs)

#特徴量となった単語たち(値は単語ID)
print(CV.vocabulary_)

#祖ベクトルを密なベクトルに変換
print(bag.toarray())

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}
[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [6]:
#TFIDFベクトル化をおこなう
from sklearn.feature_extraction.text import TfidfTransformer

TFIDF = TfidfTransformer(use_idf=True,#idf価使う
                         norm='l2',#得られたTFIDF価からなる行列を、L2ノルムで正規化する
                         #smooth_idf=True#TFIDF価の計算のとき,IDF = log((1+nd)/(1+td)) + 1←これをつけるかどうか
                         #最も頻出で全文に出てくる単語はidf=0になるが、1をつけることでTFIDFとの掛けの際に無視しないことができる
                        )

tfidfvec = TFIDF.fit_transform(bag)

#TTFIDF行列を密行列にして表示
print(tfidfvec.toarray())

[[0.         0.43370786 0.55847784 0.55847784 0.         0.43370786
  0.        ]
 [0.         0.43370786 0.         0.         0.55847784 0.43370786
  0.55847784]
 [0.40474829 0.47810172 0.30782151 0.30782151 0.30782151 0.47810172
  0.30782151]]


In [7]:
#コーパスデータを処理する前に、データのクレンジングが必要
df.loc[0,'review'][-50:]#このように、句読点やHTMLコードが入っているので、それは消す

'is seven.<br /><br />Title (Brazil): Not Available'

In [8]:
#正規表現を使って、消したい文字を消す関数
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    
    #アルファベット、アンダーバー、数字以外の文字 \W
    #については、空白置き換えののち全文小文字にする
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [9]:
#動作確認
preprocessor(df.loc[0,'review'][-50:])

'is seven title brazil not available'

In [10]:
#もとのdfのreview本文をすべてクレンジングしておく
df['review'] = df['review'].map(preprocessor)

In [11]:
#空白で分かち書きし、語幹を取り出す（ステミング）
#PorterStemmer(語幹を取り出す)
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def tokenizer(text):
    #リスト内包表記の形で、コーパスを空白区切りして単語のリストにした上で、語幹取り出し、リストにまとめる
    return [ps.stem(word) for word in text.split()]

tokenizer('They can ignore you but no one can be safe from your POWER')

['they',
 'can',
 'ignor',
 'you',
 'but',
 'no',
 'one',
 'can',
 'be',
 'safe',
 'from',
 'your',
 'power']

In [13]:
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\unlea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#コーパスをベクトル化して、ロジ回帰につっこみ、感情判定を行う
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1),(1, 2)],#unigram,,bi-gram
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, ps],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [0.01,0.1,1.0, 10.0, 100.0]},

              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


KeyboardInterrupt: 

In [15]:
#tokenizerと、csvから本文とスコアの取り出し関数を定義する
def tokenizer(text):
    text = re.sub('<[^>]*>','',text)#糞HTMLコードを削除する
    #findall(pattern,str)でstrからpatternにマッチする要素をリストの形で取得する
    #emotions = re.findall('(?::|;|=)(?:-)(?:|)(?:\)',text.lower())
    text = re.sub('[/W]+',' ',text.lower())# + ' '.join(emotions).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path,'r',encoding='utf-8') as f:
        next(f)#各ループの先頭の要素を取得するが、forloopの外なので、先頭の要素(=header)を読み飛ばす処理をしている
        for line in f:
            
            #print(line)
            text,label = line[:-3],int(line[-2])
            yield text,label
            #print(text,label)

next(stream_docs(path='C:/Users/unlea/Desktop/Python 3/ML_Master/python-machine-learning-book-3rd-edition/ch08/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [16]:
#minibatch関数を定義する
def minibatch(doc_stream,size):
    docs, y  = [], []
    
    try:
        for _ in range(size):
            text,label = next(doc_stream)#forループの中で、毎回のループの先頭の要素を取得する
            
            docs.append(text)#本文をリストに格納
            y.append(label)#スコアをリストに格納する
            
    except StopIteration:
        print('end')
        return None,None
        
    return docs,y
            

In [17]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
#hashing vectorizer
vec = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)#オレオレtokenizer

clf = SGDClassifier(loss='log',random_state=0)

doc_stream = stream_docs(path='C:/Users/unlea/Desktop/Python 3/ML_Master/python-machine-learning-book-3rd-edition/ch08/movie_data.csv')

In [18]:
#ミニバッチ手法を活用したアウトオブコア学習
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])

for _ in range(45):#４５回のループ
    
    X_train,y_train = minibatch(doc_stream,size=1000)#データ数1000のミニバッチで、45000個のデータをtrainとして順次ミニバッチ学習
    if not X_train:
        break
    X_train = vec.transform(X_train)
    
    clf.partial_fit(X_train,y_train,classes=classes)#partial_fitでデータを逐次fitさせる
    pbar.update()
    
X_test,y_test = minibatch(doc_stream,size=5000)#最後の5000個のデータで精度評価する
X_test = vec.transform(X_test)

#testデータでの精度
print('score:{}'.format(clf.score(X_test,y_test)))


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


score:0.8558


In [ ]:
#潜在ディリクレ分布(LDA,トピックモデル)について学ぶ
#LDAはの入力として、BoW行列を作成する
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#CV
Count = CountVectorizer(stop_words='english',
                        max_df=.1,#単語の最大出現頻度。全文書の１０％以上の文書に出現している単語は、頻出単語なので価値がなく、ゆえに除外する
                        max_features=5000)#BoW行列の特徴量の数を5000個に絞る

#TFIDF
tfidf = TfidfVectorizer(lowercase=True, preprocessor=None, tokenizer=tokenizer, 
                        ngram_range=(1,2),stop_words=stop,max_df=.1,max_features=5000)
#Bowにする
#X = Count.fit_transform(df['review'].values)
X = tfidf.fit_transform(df['review'].values)#TFIDF vectorization

#LDA推定器をBoW行列にFitさせる
from sklearn.decomposition import LatentDirichletAllocation#LDA
lda = LatentDirichletAllocation(n_components=10,#ハイパーパラメタとして与えるトピックの数
                                random_state=0,
                                learning_method='batch')#'online'はいわゆるミニバッチ学習。'batch'とすると、毎回のイテレーションで
                                #BoW行列をすべて利用し、パラメーターの更新を行う。
    
X_topics = lda.fit_transform(X)

#lda推定器のcomponents_属性にアクセスすると、トピック数(10)個ごとに、単語(5000words)の重要度を含んだ行列を返してくれる
print(lda.components_.shape)

In [ ]:
#10個のトピックを指定したが、10個のトピック(に分類されるとLDAが判定した文書セット)の、最も重要度の高い単語TOP5がなにか？をみてみる
n_top = 5

#feature_names = CV.get_feature_names()
feature_names = tfidf.get_feature_names()
for topic_idx,topic in enumerate(lda.components_):
    print(topic_idx + 1)
    #topicは各10トピックに分類した5000単語の重要度を格納したベクトル
    #topic.argsort()で、昇順に並べ替えたインデックスの配列を取得する
    #重要度順において昇順になっているので、argsort()[::-1]で降順にしたうえで最初の5つのindexを取得する
    print(" ".join([feature_names[i] for i in topic.argsort()[::-1][:n_top]]))